<a href="https://colab.research.google.com/github/Asuskf/download_books_springer/blob/master/download_books_springer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
beautifulsoup4 = !pip list | grep -i beautifulsoup4
googledrivedownloader = !pip list | grep -i googledrivedownloader

if len(beautifulsoup4) == 0:
  !pip install beautifulsoup4

if len(googledrivedownloader) == 0:
  !pip install googledrivedownloader
  
import requests
import shutil
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import os
from google_drive_downloader import GoogleDriveDownloader as gdd
import re

gdd.download_file_from_google_drive(file_id='1slse7jvELBMwoENYhyiUosKBcLofXpgE',
                                      dest_path='/content/libros.csv',
                                      unzip=True)

def downloadBooks (title, link, tipe):
  myBook = urlopen(link)
  pageResponse = BeautifulSoup(myBook.read(),"html5lib");
  filterResponse = [link['href'] for link in pageResponse.findAll('a', href=True) if re.findall(r'pdf$', link['href'])]
  url = 'https://link.springer.com'+filterResponse[0]
  path_books = '/content/%s'% (tipe)
  if not os.path.exists(path_books):
    os.mkdir(path_books)
  book = requests.get(url, stream=True)
  with open('%s' % (path_books+'/'+title.replace(" ", "_").replace("/", "_")+'.pdf'), 'wb') as f:
    f.write(requests.get(url).content)


books = pd.read_csv("libros.csv")
type_books = 'Intelligent Technologies and Robotics' #@param ['Behavioral Science and Psychology',  'Religion and Philosophy',  'Intelligent Technologies and Robotics',  'Computer Science',  'Business and Economics',  'Humanities, Social Sciences and Law',  'Energy',  'Mathematics and Statistics',  'Earth and Environmental Science',  'Medicine',  'Economics and Finance',  'Physics and Astronomy',  'Biomedical and Life Sciences',  'Literature, Cultural and Media Studies',  'Engineering',  'Social Sciences',  'Chemistry and Materials Science',  'Business and Management',  'Behavioral Science',  'Education',  'Law and Criminology'] {allow-input: true}
leaked_books = books[books['English Package Name'] == type_books]
books = dict(zip(leaked_books['Book Title'], leaked_books['OpenURL'].values.tolist()))
dowloads = [downloadBooks(title, link, type_books) for title, link in books.items()]
shutil.make_archive(type_books, 'zip', '/content/%s'% (type_books))
print('Descarga completa')
